# Data Exploration

Was able to find data on NYT crosswords which is what we will focus on. Data has been [uploaded to Google Drive](https://drive.google.com/drive/folders/101KDJ5dl_cSPPZPBMBKIpy5-qbN6HkBN?usp=share_link), but is intentionally private (for now).

The first set of data we will look at is a collection of clues and their respective answers.

In [99]:
import pandas as pd

df = pd.read_csv("data/nytcrosswords.csv")
df

,Date,Word,Clue
0,10/31/2021,PAT,"Action done while saying ""Good dog"""
1,10/31/2021,RASCALS,Mischief-makers
2,10/31/2021,PEN,It might click for a writer
3,10/31/2021,SEP,Fall mo.
4,10/31/2021,ECO,Kind to Mother Nature
...,...,...,...
781568,11/21/1993,NAT,Actor Pendleton
781569,11/21/1993,SHRED,Bit
781570,11/21/1993,NEA,Teachers' org.
781571,11/21/1993,BEG,Petition


In [52]:
df[df['Word'] == 'ABRAHAMLINCOLN']

,Date,Word,Clue
355023,2/12/2009,ABRAHAMLINCOLN,Notable born 2/12/1809
691197,2/12/1997,ABRAHAMLINCOLN,"Subject of Copland's ""Portrait"""
703805,9/1/1996,ABRAHAMLINCOLN,"""If I were two-faced, would I be wearing this ..."
739500,5/21/1995,ABRAHAMLINCOLN,==}:?�]]


## Look for bad data
Let's look for missing data, or other weirdness we want to remove from the dataset. We can start with the obvious: empty values.

In [2]:

df['word_length'] = df['Word'].apply(lambda x: len(str(x).strip()))
df['clue_length'] = df['Clue'].apply(lambda x: len(str(x).strip()))

In [3]:
df[df['word_length'] <= 1]

,Date,Word,Clue,word_length,clue_length
223435,10/31/2013,,"64-Across, in 23-Down",0,21


In [4]:
df[df['clue_length'] <= 0]

,Date,Word,Clue,word_length,clue_length


In [5]:
print(f'Number of nulls: {len(df[df["Word"].isnull()])}')
df[df["Word"].isnull()].head()

Number of nulls: 34


,Date,Word,Clue,word_length,clue_length
24006,12/24/2020,NaN,Zero,3,4
43275,4/17/2020,NaN,Empty,3,5
67323,6/7/2019,NaN,Of no value,3,11
93805,6/24/2018,NaN,No longer in force,3,18
141809,10/1/2016,NaN,"{ }, in mathematics",3,19


In [6]:
print(f'Number of nulls: {len(df[df["Clue"].isnull()])}')
df[df["Clue"].isnull()].head()

Number of nulls: 0


,Date,Word,Clue,word_length,clue_length


After grepping the csv manually it turns out that these `NaN` values are actually the string "NULL" in the dataset which causes parsing issues. Something we'll have to solve with the cleanup.

So far, we know that we must:
1. Remove empty rows
2. Replace `NaN` with "NULL"

Next, let's make sure that we don't have any weird characters.

In [7]:
allowed_chars = r'^[A-Za-z\s\."-_!]+$'
mask = ~df['Clue'].str.contains(allowed_chars)
df[mask]

,Date,Word,Clue
80,10/31/2021,PEPE,Nickname for Josﺅﺟﺛ
396,10/27/2021,ONE,What the tangent of 45ﺅﺟﺛ is equal to
465,10/26/2021,SNERT,Hﺅﺟﺛgar the Horrible's dog
812,10/21/2021,STAG,Animal on a Jﺅﺟﺛgermeister bottle
981,10/19/2021,ACUTEACCENT,Bit of dﺅﺟﺛcor?
...,...,...,...
781041,11/27/1993,DEBASER,He?ﺅﺟﺛll humiliate you
781050,11/27/1993,ASI,?ﺅﺟﺛ___ was saying...?ﺅﺟﺛ
781054,11/27/1993,NOTEPAD,Reporter?ﺅﺟﺛs book
781062,11/27/1993,SAWDUST,Carpenter?ﺅﺟﺛs floor cover


Seems like we have an encoding issue. Let's dig into that.

In [76]:
import chardet
with open("data/nytcrosswords.csv", 'rb') as file:
    print(chardet.detect(file.read()))

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


We know the encoding isn't utf-8, so let's keep trying.

In [12]:
with open("data/nytcrosswords.csv") as file:
    print(file)

<_io.TextIOWrapper name='data/nytcrosswords.csv' mode='r' encoding='UTF-8'>


Same problem. Found another library that might be helpful - couldn't get this installed with conda, so had to do a `pip install charamel`.

In [38]:
from charamel import Detector
detector = Detector()
with open("data/nytcrosswords.csv") as file:
    print(detector.probe(file.read(), top=10))

[(<Encoding.CP_1006: 'cp1006'>, 0.9521461826551444), (<Encoding.CP_864: 'cp864'>, 0.9462450387005286), (<Encoding.UTF_7: 'utf_7'>, 0.9452766125829656), (<Encoding.ISO_2022_JP: 'iso2022_jp'>, 0.9226919055739324), (<Encoding.ISO_2022_KR: 'iso2022_kr'>, 0.9222854034560479), (<Encoding.ASCII: 'ascii'>, 0.9220496290394135), (<Encoding.ISO_2022_JP_2004: 'iso2022_jp_2004'>, 0.92202299533934), (<Encoding.ISO_2022_JP_3: 'iso2022_jp_3'>, 0.9219871621048285), (<Encoding.HZ: 'hz'>, 0.9218368758036212), (<Encoding.ISO_2022_JP_2: 'iso2022_jp_2'>, 0.9207895019977104)]


Progress! Let's try with these encodings.

In [78]:
test_encoding_df = pd.read_csv("data/nytcrosswords.csv")

allowed_chars = r'^[A-Za-z\s\."-_!]+$'
mask = ~test_encoding_df['Clue'].str.contains(allowed_chars)
test_encoding_df[mask]

,Date,Word,Clue
80,10/31/2021,PEPE,Nickname for Jos�
396,10/27/2021,ONE,What the tangent of 45� is equal to
465,10/26/2021,SNERT,H�gar the Horrible's dog
812,10/21/2021,STAG,Animal on a J�germeister bottle
981,10/19/2021,ACUTEACCENT,Bit of d�cor?
...,...,...,...
781041,11/27/1993,DEBASER,He?�ll humiliate you
781050,11/27/1993,ASI,?�___ was saying...?�
781054,11/27/1993,NOTEPAD,Reporter?�s book
781062,11/27/1993,SAWDUST,Carpenter?�s floor cover


After trying all the encodings I can think of _and_ using some offline tools I'm still stumped. Let's just manually clean up the rows we can and drop the rest.

In [128]:
df.at[80, 'Clue'] = 'Nickname for Jose'
df.at[396, 'Clue'] = 'What the tangent of 45° is equal to'
df.at[465, 'Clue'] = 'Hagar the Horrible\'s dog'
df.at[812, 'Clue'] = 'Animal on a Jagermeister bottle'
df.at[981, 'Clue'] = 'Bit of decor?'

# from chat gpt
df.at[1111, 'Clue'] = 'Part of the body named after Dr. Ernst Grafenberg'
df.at[1147, 'Clue'] = 'Sight on an M. C. Escher Mobius strip'
df.at[1238, 'Clue'] = 'Its anthem is "Terre de nos aieux"'
df.at[1473, 'Clue'] = 'Opera division'
df.at[1476, 'Clue'] = 'Reunion, for one'
df.at[1624, 'Clue'] = 'Magazine co-founded in 1945 by Helene Gordon'
df.at[2212, 'Clue'] = 'AZUR Cote d\'Azur'
df.at[2282, 'Clue'] = 'Eleven, en francais'
df.at[2292, 'Clue'] = ' ___ d\'Etat'
df.at[2446, 'Clue'] = 'Hagar the Horrible\'s pooch'
df.at[2557, 'Clue'] = 'Like Prince William vis-a-vis Prince Harry'
df.at[2579, 'Clue'] = 'One of a resume pair?'
df.at[2701, 'Clue'] = 'Function whose output is 45° when applied to 1'
df.at[2881, 'Clue'] = 'The singer Bjork, for one'
df.at[2937, 'Clue'] = '£ : pound sterling :: R : ___'
df.at[2940, 'Clue'] = '58 ___ Mediterraneo'
df.at[3024, 'Clue'] = 'Ile de la Cite, one end of Paris\'s Pont Neuf'
df.at[3091, 'Clue'] = 'Fish often served "meuniere"'

df.at[3254, 'Clue'] = 'Wielder of the hammer Mjölnir'
df.at[3402, 'Clue'] = 'Quinceañeras, e.g.'
df.at[3450, 'Clue'] = 'Perfect vis-à-vis good, in an aphorism'
df.at[3630, 'Clue'] = 'Risqué'
df.at[4037, 'Clue'] = 'Emily Brontë pseudonym'
df.at[4056, 'Clue'] = "Aéroport d'Orly"
df.at[4407, 'Clue'] = '"90 Day Fiancé" channel'
df.at[4459, 'Clue'] = '0, for 0°'
df.at[4618, 'Clue'] = 'Word on either side of "à"'
df.at[5558, 'Clue'] = 'Opposite of là'
df.at[5883, 'Clue'] = '365 días'
df.at[6215, 'Clue'] = 'José Olé (frozen foods brand)'
df.at[6347, 'Clue'] = 'Tête-à-tête'
df.at[6409, 'Clue'] = 'What good soufflés do'
df.at[6410, 'Clue'] = '"Liberté, égalité, fraternité" (Haiti\'s motto)'
df.at[6421, 'Clue'] = 'Les États-Unis'
df.at[6571, 'Clue'] = '"De dónde eres?" (Spanish 101 query)'

allowed_chars = r'^[A-Za-z\s\."-_!£öñàéë°íê]+$'
mask = ~df['Clue'].str.contains(allowed_chars)
print(len(df[mask]))
df[mask].head(20)

1238


,Date,Word,Clue
4947,8/28/2021,RURAL,Like many scenes of the artist Jean-François M...
6421,8/9/2021,LES,Les États-Unis
6571,8/7/2021,ERES,"""De dónde eres?"" (Spanish 101 query)"
6623,8/6/2021,SHAME,"""___ has poor memory"": Gabriel Garc�a M�rquez"
6696,8/5/2021,ESO,"That, en Espa�a"
7069,8/1/2021,ELIO,"___ Perlman, role for Timoth�e Chalamet in ""Ca..."
7102,7/31/2021,AZTEC,Like the 16th-century ruler Cuauht�moc
7174,7/30/2021,GRASSHOPPER,"Southern cocktail made with cr�me de menthe, c..."
7321,7/28/2021,LEAPS,"Jet�s, e.g."
7328,7/28/2021,EVA,Se�ora Per�n


Using ChatGPT the missing characters can be identified pretty easily, but it's still a huge pain to update them and I think the special characters will make our life more difficult in the future so let's just ignore them and reconsider this in the future if we need to.

We now need to:
* Remove empty rows
* Replace NaN with "NULL"
* Remove rows with characters that we don't like

Let's do this in a new notebook and then take a fresh look at the data.

## Future cleaning to consider
* unknown characters
* "see 64-across", {See Notepad}, etc ex